# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} # we want the output to be a JSON object - required by OpenAI but not by Claude
    )
    result = response.choices[0].message.content
    return json.loads(result)

response.choices[0] --> choice of the response returned. There may be variations in the choices which are in a list format

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/Janus-Pro-7B',
 '/mistralai/Mistral-Small-24B-Instruct-2501',
 '/deepseek-ai/DeepSeek-V3',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Qwen/Qwen2.5-Max-Demo',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces',
 '/datasets/cognitivecomputations/dolphin-r1',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/ServiceNow-AI/R1-Distill-SFT',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarl

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    # parse in a webpage - get the relevant links and put those webpages below in the same block of text - the result object
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url) # calling to gpt4-mini
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-R1
Updated
5 days ago
•
1.23M
•
7.09k
deepseek-ai/Ja

In [14]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n5 days ago\n•\n1.23M\n•\n7.09k\ndeepseek-ai/Janus-Pro-7B\nUpdated\n5 days ago\n•\n194k\n•\n2.64k\nmistralai/Mistral-Small-24B-Instruct-2501\nUpdated\n4 days ago\n•\n24k\n•\n615\ndeepseek-ai/DeepSeek-V3\nUpdated\n13 days ago\n•\n984k\n•\n3.18k\nhexgrad/Kokoro-8

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL, # replace 4o with o1-mini or other models
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)} # calls a function which calls gpt4o
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Unleash the Cuteness of AI!

## Welcome to Hugging Face!

### Where AI Meets Affection
At Hugging Face, we're not just building AI; we're building a community that's as warm and fuzzy as our name suggests. Welcome to the future of machine learning, where we brainstorm models, datasets, and applications faster than you can say “artificial intelligence is more than just complicated math!”

### Our Customers: A Star-Studded Lineup!
Over **50,000 organizations** are cuddling up with us! Our clients range from **Google**, **AI at Meta**, **Amazon Web Services**, to **Microsoft**—basically, the A-Team of tech! We might not be giving them actual hugs, but we provide the next best thing: high-performance AI models and data galore!

### What We Offer: Models, Datasets & More!
- **1M+ Models**: Looking for a model to wear? We don’t have fashion models, but we do have AI models ready to strut their stuff!
- **250k+ Datasets**: Our datasets are like a treasure trove waiting to be discovered! (Shovel not included.) 
- **Spaces**: It’s not just ‘Outer Space’ – it’s our collaborative playground where you can launch your AI creations into the universe!

### Our Culture: Open, Friendly, and Slightly Nerdy
At Hugging Face, we believe in **collaboration**, **openness**, and **a good cup of coffee**. Join our team of AI enthusiasts who prefer hugging models to hugging trees. Our culture is as vibrant as our codebase; think **hackathons** where high-fives are mandatory and the only time you’ll hear “you can’t do that” is during flip cup!

### Careers: Join the Hug Life
Are you looking for a job that’s as fun as a puppy pile? We’re on the lookout for:
- **AI Magicians**: Turn data into gold—figuratively, of course.
- **Model Whisperers**: For when model training becomes a therapy session.
- **Data Ninjas**: Stealthily navigating through datasets like a pro!

### Pricing: Pay Less, Hug More!
- **Compute**: Starting at just **$0.60/hour for GPU!** (That's like, cheaper than your morning coffee at that hipster café! ☕)
- **Enterprise Solutions**: From **$20/user/month**, because AI and a cozy hug don't have to break the bank!

### Join Us!
Are you ready to embrace the Hugging Face revolution? With a mission to make machine learning more accessible than ever, let’s take the leap together! Whether you’re looking to innovate, invest, or grab a coding job, we’ve got a space for you.

---

### Visit us at [Hugging Face](https://huggingface.co/) to join the snuggly world of AI! 🌍💕

---
> “Never underestimate the power of a hug… especially if it’s a Hugging Face AI model ready to rock the world!”

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True # results flow back - each chunk of the response flows through
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face: Where AI Gets *Hugged* with Love 🤗

Welcome to the plush, cuddly world of Hugging Face, the AI community that’s building the future one snuggle at a time! Think of us as the teddy bear of tech—soft on the outside, but packed with powerful algorithms and machine learning treasures inside! 

## Why Hugging Face? 
- **We're Hugging the Future**: We believe that the best things in life begin with a hug. And by "hug", we mean a beautiful collaboration among the brightest minds in AI! Join us as we reshape the digital world, one cozy project at a time!
  
## What We Offer:
### 1. **Models** 🧠
With over **1M+ models** and counting, we provide more variations than your grandma’s cookie jar. Need a model? We’ve got deepseek, mistral, and all the amazing categories you can dream of - we even have a model called “Janus-Pro-7B”. Sounds like a superhero, right? 

### 2. **Datasets** 📚
Our datasets are as diverse as your own book collection (minus that weird one you’ve been too embarrassed to read). With **250k+ datasets**, you're bound to find something useful or at least entertaining!

### 3. **Spaces** 🌌
Thanks to *Spaces*, you can interact with AI without the risks of awkward small talk. We've got chatbots, text-to-3D miracles, and more—no skills required, just a love for AI! 

### 4. **Enterprise Solutions** 🏢
Looking to give your company a techie glow-up? With enterprise-grade security and tools, we’re the fairy godmothers of AI - turning pumpkins into powerful models! Just remember, this magic DOES come with pricing (and, unfortunately, no glass slippers).

## Our Community 👥
Over **50,000 organizations** (yes, we counted them!) love us, including giants like Amazon Web Services, Google, and Microsoft. You know you’re in good company when the WHOLE OF AI CANON is taking note!

## Culture: We’re a Friendly Bunch! 🌈
Forget the stiff-collared, stale coffee atmosphere. At Hugging Face, laugh tracks are optional but encouraged! Our team embodies:
- **Collaboration**: You bring the ideas, we bring the hugs. 
- **Innovation**: Where "that would be cool" becomes "let’s build that!" 
- **Diversity**: We believe diversity in thought is the secret recipe for innovation. 

We vibe on humor and creativity as we tackle the intense world of AI—who said tech couldn't be fun?

## Careers: Want to Join the Hug Fest? 🤗
If you want to expand your horizons while smothering your colleagues in job satisfaction and cuddly algorithms, check our careers page! We're continuously on the lookout for passionate, quirky, and innovative individuals (bonus if you can tell us your favorite AI pun). 

## Join the Hugging Face Family! 🌍 
Whether you’re here to code, create, or collaborate, Hugging Face is your happy place! Join us to be a part of something BIG, HUGGABLE, and might we say, utterly transformational. 

### Ready to Hug It Out? 
Explore more at **[huggingface.co](https://huggingface.co)**—where the future is friendly, fluffy, and fabulously fun! 

--- 

*Disclaimer: Standing too close to Hugging Face may result in spontaneous collaboration, laughter, and a strong desire for group hugs. Proceed with joy!*


In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## Welcome to Hugging Face!
Hugging Face is a leading platform at the forefront of the AI community, dedicated to building the future of machine learning (ML). Our mission is to create an open, collaborative environment where individuals and organizations can come together to share knowledge, build models, and advance the capabilities of AI.

### What We Offer

- **Models**: Access to over **1 million models** that support cutting-edge algorithms and technologies in natural language processing, computer vision, and more.
- **Datasets**: Browse through more than **250,000 datasets** to fuel your experiments and innovations.
- **Spaces**: Deploy your machine learning applications effortlessly with **400,000+ applications** running in our collaborative environment.
- **Enterprise Solutions**: Tailored solutions for businesses that need enterprise-grade security, access controls, and dedicated support.

### Our Culture
At Hugging Face, we believe in the power of community and collaboration. Our culture fosters inclusivity and encourages innovation. We are committed to open-source values, enabling everyone—from researchers to students—to access ML tools. We prioritize learning, sharing, and growing together as a community.

### Our Customers
We proudly serve a diverse set of over **50,000 organizations**, including industry giants such as:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Grammarly**

### Careers at Hugging Face
Join us in our mission! Hugging Face offers exciting career opportunities across various roles. We seek passionate individuals who thrive in a collaborative environment and are eager to contribute to the AI community. Discover open positions [here](#).

### Contact Us
Want to learn more about Hugging Face or get involved? Reach out to us:
- **GitHub**: [Hugging Face](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face](https://www.linkedin.com/company/huggingface)

Join Hugging Face and be part of the revolutionary movement in AI. **Together, we’re building the future!**

---

For more details, visit our website at [huggingface.co](https://huggingface.co)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>